In [ ]:
# INSTALLS

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 




E: Unable to locate package gdal-bin
E: Unable to locate package python-gdal
E: Unable to locate package python3-gdal



E: Unable to locate package python3-rtree
  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-51pwf2nw
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-51pwf2nw
  Created wheel for geopandas: filename=geopandas-0.9.0+12.gafa49e0-py2.py3-none-any.whl size=995084 sha256=2ee194cb4354381fa2d873302eebcc0e4e3ee6dc372d0bb32f8ca9b50c53175a
  Stored in directory: /tmp/pip-ephem-wheel-cache-osfbn9sr/wheels/cf/3e/0b/6475054094c2b1ea054158ac1fdcf749fb92f5b512377e4cf8
Successfully built geopandas
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [402]:
# IMPORTS

import pandas as pd
import numpy as np
import os
import geopandas as gpd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [5]:
# MOUNT DRIVE

root_dir = '/datasets/datathondeepnoteaccess/'
base_dir = root_dir + 'Stanford_Datathon_103/'  

In [6]:
# READ IN THE DATA

# Pre-processed data from 
# https://opendoorsdata.org/data/international-students/leading-places-of-origin/
# https://opendoorsdata.org/data/us-study-abroad/all-destinations/ 

file1 = base_dir + 'dataset/intStdnts_inUS_origin.csv'
file2 = base_dir + 'dataset/USStdnts_abroad.csv'
data_in = pd.read_csv(file1)
data_out = pd.read_csv(file2)

# CLEAN DATA

In [12]:
data_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Place of Origin  500 non-null    object 
 1   Int'l Students   500 non-null    float64
 2   % of Total       500 non-null    float64
 3   Year             500 non-null    object 
dtypes: float64(2), object(2)
memory usage: 15.8+ KB


In [13]:
# Rename columns
data_in_1 = data_in.rename(columns={'Place of Origin':'country',
                                    'Int\'l Students':'intStdnts',
                                    '% of Total':'percTotal',
                                    'Year':'year'})


# data_in_1["intStdnts"] = data_in_1["intStdnts"].str.replace(",","").astype(float)
data_in_1["percTotal"] = data_in_1["percTotal"]/100.0
data_in_1['year'] = data_in_1.year.str.extract('(\d+)').astype(int)

data_in_1['country'] = data_in_1['country'].str.replace("China, PRC","China")
data_in_1['country'] = data_in_1['country'].str.replace("Hong Kong, China","Hong Kong")

data_in_1.info()
data_in_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   country    500 non-null    object 
 1   intStdnts  500 non-null    float64
 2   percTotal  500 non-null    float64
 3   year       500 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 15.8+ KB


,country,intStdnts,percTotal,year
0,China,59939.0,0.109,2000
1,India,54664.0,0.100,2000
2,Japan,46497.0,0.085,2000
3,South Korea,45685.0,0.083,2000
4,Taiwan,28566.0,0.052,2000


In [14]:
data_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Destinations  222 non-null    object
 1   Region        222 non-null    object
 2   Continent     222 non-null    object
 3   1999/00       222 non-null    object
 4   2000/01       222 non-null    object
 5   2001/02       222 non-null    object
 6   2002/03       222 non-null    object
 7   2003/04       222 non-null    object
 8   2004/05       222 non-null    object
 9   2005/06       222 non-null    object
 10  2006/07       222 non-null    object
 11  2007/08       222 non-null    object
 12  2008/09       222 non-null    object
 13  2009/10       222 non-null    object
 14  2010/11       222 non-null    object
 15  2011/12       222 non-null    object
 16  2012/13       222 non-null    object
 17  2013/14       222 non-null    object
 18  2014/15       222 non-null    object
 19  2015/16 

In [15]:
# data_out.head()

# Rename columns
data_out_1 = data_out.rename(columns={'Destinations':'country',
                                    'Region':'region',
                                    'Continent':'continent',
                                    '1999/00':'1999','2000/01':'2000','2001/02':'2001','2002/03':'2002',
                                    '2003/04':'2003','2004/05':'2004','2005/06':'2005','2006/07':'2006',
                                    '2007/08':'2007','2008/09':'2008','2009/10':'2009','2010/11':'2010',
                                    '2011/12':'2011','2012/13':'2012','2013/14':'2013','2014/15':'2014',
                                    '2015/16':'2015','2016/17':'2016','2017/18':'2017','2018/19':'2018'})

data_out_1 = data_out_1.drop(['region','continent'], axis=1)

# data_out_1.set_index('country')

all_years = ['1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']
data_out_1[all_years] = data_out_1[all_years].replace("-","0")
data_out_1[all_years] = data_out_1[all_years].apply(pd.to_numeric, errors='coerce')


data_out_1.info()
data_out_1.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  222 non-null    object
 1   1999     222 non-null    int64 
 2   2000     222 non-null    int64 
 3   2001     222 non-null    int64 
 4   2002     222 non-null    int64 
 5   2003     222 non-null    int64 
 6   2004     222 non-null    int64 
 7   2005     222 non-null    int64 
 8   2006     222 non-null    int64 
 9   2007     222 non-null    int64 
 10  2008     222 non-null    int64 
 11  2009     222 non-null    int64 
 12  2010     222 non-null    int64 
 13  2011     222 non-null    int64 
 14  2012     222 non-null    int64 
 15  2013     222 non-null    int64 
 16  2014     222 non-null    int64 
 17  2015     222 non-null    int64 
 18  2016     222 non-null    int64 
 19  2017     222 non-null    int64 
 20  2018     222 non-null    int64 
dtypes: int64(20), object(1)
memory usage: 3

,country,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Burundi,0,0,0,0,0,1,0,1,0,...,0,2,9,0,1,2,0,11,0,0
1,Djibouti,0,0,0,0,0,0,0,0,0,...,23,1,4,1,1,0,0,0,0,0
2,Eritrea,0,18,0,12,1,14,8,1,0,...,3,0,1,0,1,0,1,1,0,5
3,Ethiopia,0,12,30,26,26,33,56,84,115,...,191,147,267,231,235,261,302,73,207,249
4,Kenya,695,846,720,625,387,661,694,686,657,...,1198,1291,1231,1238,1022,634,681,778,927,926


# VIZ

## Incoming students over time

In [396]:

dat_vz_incoming = data_in_1[['country','intStdnts','year']].query('intStdnts >= 1000')
# dat_vz_incoming.info()

table_i = dat_vz_incoming.pivot_table(index='year', columns='country', values='intStdnts').reset_index()
# table.head()
# print(table)

fig = px.line(table_i,x='year', y=table_i.columns[:])
fig.show()

## Outgoing students over time

In [493]:
# data_out_1.head()
dat_vz_outoing = data_out_1.set_index('country').transpose().reset_index()
dat_vz_outoing = dat_vz_outoing.rename(columns={'index':'year'})

# dat_vz_outoing.head()
# dat_vz_outoing.info()

fig = px.line(dat_vz_outoing,x='year', y=dat_vz_outoing.columns[:])
fig.show()

## Map for 1 country, incoming vs outgoing students, over time

In [525]:
c = 'Brazil'
combine_i = table_i[[c,'year']].set_index('year')
combine_i.drop([2019], inplace=True)
combine_i = combine_i.rename(columns={c:'incoming'})
# combine_i.tail()

combine_o = dat_vz_outoing[[c, 'year']]
combine_o['year'] = pd.to_numeric(combine_o['year'])
combine_o = combine_o.set_index('year')
combine_o.drop([1999], inplace=True)
combine_o = combine_o.rename(columns={c:'outgoing'})
# combine_o.head()

combine = pd.concat([combine_i,combine_o], axis=1).reset_index()
# print(combine)

# fig = px.line(combine,x='year', y=combine.columns[:], title='Students from '+c+' studying in the US vs students from the US studying in '+c)
fig = px.line(combine,x='year', y=combine.columns[:], title=c)
fig.update_layout(title_x=0.45, title_y=0.75, width=400, height=300, legend_title="Flow of students", )

fig.show()


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Subplots

## 25 Countries on worldmap, ratio of incoming:outgoing students

In [153]:
# dataframe for countries, last available year: incoming | outgoing | ratio
data_out_2 = data_out_1.set_index('country').transpose().reset_index()
data_out_2 = data_out_2.rename(columns={'index':'year'})
# data_out_2.info()

last_year_in = int(data_in_1['year'].max())
last_year_out = int(data_out_2['year'].max())
pick_year = min(last_year_in, last_year_out)
print("latest data for incoming students: ",last_year_in)
print("latest data for outgoing students: ",last_year_out)
print("pick: ", pick_year)

last_year = data_in_1.query('year == 2018')[['country','intStdnts',]]
last_year = last_year.set_index('country')
last_year = last_year.rename(columns={'intStdnts':'incoming'})

# # .query('intStdnts >= 1000')

for index, row in last_year.iterrows():
  try: 
    last_year.at[index,'outgoing']= data_out_1.set_index('country')['2018'][index]
    last_year.at[index,'ratioIn2Out'] = last_year.at[index,'incoming'] / last_year.at[index,'outgoing']
  except Exception as e:
    last_year.at[index,'outgoing'] = 0
    last_year.at[index,'ratioIn2Out'] = 0

c = 'China'
# print("Data for ", c, " is ",data_out_1.set_index('country')['2018'][c])
# data_out_1.head()
last_year.head()

latest data for incoming students:  2019
latest data for outgoing students:  2018
pick:  2018


,incoming,outgoing,ratioIn2Out
country,,,
China,369548.0,11639.0,31.750838
India,202014.0,3366.0,60.016043
South Korea,52250.0,4558.0,11.463361
Saudi Arabia,37080.0,4.0,9270.000000
Canada,26122.0,2107.0,12.397722


In [524]:
# PLOTLY MAP Choropleth Maps  https://plotly.com/python/choropleth-maps/ 
map_data = last_year.reset_index()
# map_data = map_data.drop(['overlayTxt'], axis=1)


for index, row in map_data.iterrows():
  try: 
    map_data.at[index,'overlayTxt'] = '%s <br><br>incoming: %d<br>outgoing: %d<br>ratio: %3.2f' % (map_data.at[index,'country'],map_data.at[index,'incoming'],map_data.at[index,'outgoing'],map_data.at[index,'ratioIn2Out'])
    # map_data.at[index,'overlayTxt'] = '%s <br>incoming: %d<br>outgoing: %d<br>ratio: %d' % (map_data.at[index,'country'], map_data.at[index,'incoming'], map_data.at[index,'outoing'], map_data.at[index,'ratioIn2Out'])
  except Exception as e:
    print('error', map_data.at[index,'country'])
    # last_year.at[index,'overlayTxt'] = map_data.at[index,'country']
# map_data.head()

map_data = map_data.reset_index()
# print(map_data['country'].tolist())

max_ratio = int(map_data['ratioIn2Out'].max())
print(max_ratio)

values = [i for i in range(-1,5)]
ticks = ['%d : 1' % 10**i for i in values]

data = dict (
    type = 'choropleth',
    locations = map_data['country'],
    locationmode='country names',
    # colorscale = ['Viridis'],
    text = map_data['overlayTxt'],
    hoverinfo="text",
    colorscale = 'bluyl',
    marker_line_color='darkgray',
    marker_line_width=0.1,
    colorbar = {'title' : 'Ratio of <br>incoming <br>international <br>vs outoing <br>US students', 
    'tickvals':values, 'ticktext':ticks, 'len':0.75 },
    z=np.log10(map_data['ratioIn2Out']))

map = go.Figure(data=[data])



# print(values)
# print(ticks)

map.update_geos(
    showcountries=False,
    # projection_type="natural earth"
)

map.update_layout(
    title_text='Geographic patterns of ratio between incoming international vs outgoing US students',
    title_x=0.5,
    title_y=0.85,
    width = 800,
    margin={"r":0,"t":50,"l":0,"b":0}
)


map.show()

9270


# DJW

## Plot Inflow/Outflow

In [412]:
dw_in = data_in_1.query('year == 2018').copy(deep=True)
dw_in.head()

,country,intStdnts,percTotal,year
450,China,369548.0,0.337,2018
451,India,202014.0,0.184,2018
452,South Korea,52250.0,0.048,2018
453,Saudi Arabia,37080.0,0.034,2018
454,Canada,26122.0,0.024,2018


In [421]:
dw_out = data_out_1[['country','2018']].copy(deep=True)
# data_out_1.set_index('country')['2018'][index]
dw_out.head()

,country,2018
0,Burundi,0
1,Djibouti,0
2,Eritrea,5
3,Ethiopia,249
4,Kenya,926


In [422]:
dw_in['direction'] = 'incoming'
dw_in['students'] = dw_in.intStdnts
dw_in.head()

,country,intStdnts,percTotal,year,direction,students
450,China,369548.0,0.337,2018,incoming,369548.0
451,India,202014.0,0.184,2018,incoming,202014.0
452,South Korea,52250.0,0.048,2018,incoming,52250.0
453,Saudi Arabia,37080.0,0.034,2018,incoming,37080.0
454,Canada,26122.0,0.024,2018,incoming,26122.0


In [423]:
# convert out df from wide to long
dw_out = dw_out.melt(id_vars='country', var_name='year', value_name='students')


In [424]:
# add outgoing label
dw_out['direction'] = 'outgoing'

In [425]:
dw_out.head()

,country,year,students,direction
0,Burundi,2018,0,outgoing
1,Djibouti,2018,0,outgoing
2,Eritrea,2018,5,outgoing
3,Ethiopia,2018,249,outgoing
4,Kenya,2018,926,outgoing


In [426]:
# 32 countries in
countries_in = np.unique(dw_in.country)
print(f'{len(np.unique(countries_in))} incoming countries...')
# 222 countries out
countries_out = np.unique(dw_out.country)
print(f'{len(np.unique(countries_out))} outgoing countries...')
# overlap?
print('\nCountries in both sets:')
print(countries_in[np.in1d(countries_in,countries_out)])
print(f'\n{len(countries_in[np.in1d(countries_in,countries_out)])} countries in both...')


25 incoming countries...
222 outgoing countries...

Countries in both sets:
['Bangladesh' 'Brazil' 'Canada' 'China' 'Colombia' 'France' 'Germany'
 'India' 'Indonesia' 'Iran' 'Japan' 'Kuwait' 'Malaysia' 'Mexico' 'Nepal'
 'Nigeria' 'Pakistan' 'Saudi Arabia' 'South Korea' 'Spain' 'Taiwan'
 'Turkey' 'United Kingdom' 'Venezuela' 'Vietnam']

25 countries in both...


In [427]:
# make out same as in countries
countries = countries_in[np.in1d(countries_in,countries_out)]
dw_out = dw_out.loc[dw_out.country.isin(countries),:].reset_index(drop=True)
dw_out

,country,year,students,direction
0,Nigeria,2018,38,outgoing
1,China,2018,11639,outgoing
2,Japan,2018,8928,outgoing
3,South Korea,2018,4558,outgoing
4,Taiwan,2018,1270,outgoing
5,Bangladesh,2018,45,outgoing
6,India,2018,3366,outgoing
7,Nepal,2018,647,outgoing
8,Pakistan,2018,7,outgoing
9,Indonesia,2018,677,outgoing


In [428]:
# merge the 32 countries for both
dw_in_out = pd.concat([dw_in, dw_out], ignore_index=True, sort=False)

In [429]:
dw_in_out

,country,intStdnts,percTotal,year,direction,students
0,China,369548.0,0.337,2018,incoming,369548.0
1,India,202014.0,0.184,2018,incoming,202014.0
2,South Korea,52250.0,0.048,2018,incoming,52250.0
3,Saudi Arabia,37080.0,0.034,2018,incoming,37080.0
4,Canada,26122.0,0.024,2018,incoming,26122.0
5,Vietnam,24392.0,0.022,2018,incoming,24392.0
6,Taiwan,23369.0,0.021,2018,incoming,23369.0
7,Japan,18105.0,0.017,2018,incoming,18105.0
8,Brazil,16059.0,0.015,2018,incoming,16059.0
9,Mexico,15229.0,0.014,2018,incoming,15229.0


### Stacked Bar

In [430]:
dw_group = pd.DataFrame(dw_in_out.groupby(['country', 'direction'])['students'].mean()).reset_index()
dw_group = dw_group.sort_values(by="students", ascending=False)

In [490]:
fig = px.bar(dw_group, y="country", x="students", color="direction", orientation="h",
    labels=dict(country="Top 25 countries sending students to the US", students="International Students in 2018/2019", direction="Flow of students"))

fig.update_layout(title_text='Inflows/outflows comparison of international students by country', 
title_x=0.5, width=700, yaxis=dict(autorange="reversed") )
fig.show()

In [502]:
fig = px.bar(dw_group, x="country", y="students", color="direction", title="Inflows/Outflows")
fig.show()

In [510]:
dw_group.loc[dw_group.country=="Spain"]

,country,direction,students
39,Spain,outgoing,33849.0
38,Spain,incoming,7262.0


In [512]:
33849/7262

4.66111264114569

### Marimekko

In [358]:
# create top 15 out and extract incoming from mean grouped

out_mean = dw_group.loc[dw_group.direction=='outgoing']
out_mean.sort_values(by='students', ascending=False)
out15_mean = out_mean[0:15]
out15_mean = out15_mean.rename(columns={'students': 'students_out'}).drop(columns='direction')

in_mean = dw_group.loc[dw_group.direction=='incoming']
in_mean = in_mean.rename(columns={'students': 'students_in'}).drop(columns='direction')

In [362]:
# combine in one df
dw_mean15 = pd.merge(out15_mean, in_mean, on='country')

In [369]:
# calculations for marimekko
dw_mean15['total_flow'] = dw_mean15.students_in + dw_mean15.students_out
dw_mean15['prop_in'] = (dw_mean15.students_in/dw_mean15.total_flow)*100
dw_mean15['prop_out'] = (dw_mean15.students_out/dw_mean15.total_flow)*100
total_flow = dw_mean15.students_in.sum() + dw_mean15.students_out.sum()
dw_mean15['prop_total'] = (dw_mean15.total_flow/total_flow)*100

In [387]:
# sort
dw_mean15 = dw_mean15.sort_values(by='prop_out', ascending=False)

In [393]:
labels = list(dw_mean15.country)
widths = np.array(dw_mean15.prop_total)

students = {
    "Outgoing": list(dw_mean15.students_out),
    "Incoming": list(dw_mean15.students_in)
}

data = {
    "Outgoing": list(dw_mean15.prop_in),
    "Incoming": list(dw_mean15.prop_out)
}

fig = go.Figure()
for key in data:
    fig.add_trace(go.Bar(
        name=key,
        y=data[key],
        x=np.cumsum(widths)-widths,
        width=widths,
        offset=0,
        customdata=np.transpose([labels, widths*data[key]]),
        #texttemplate="%{y} x %{width} =<br>%{customdata[1]}",
        #textposition="inside",
        #textangle=0,
        #textfont_color="white",
        hovertemplate="<br>".join([
            "%{customdata[0]}",
            "% of total: %{width:.1f}",
            "% of students: %{y:.1f}",
        ])
    ))

fig.update_xaxes( 
    tickvals=np.cumsum(widths)-widths/2, 
    ticktext= ["%s" % (l) for l in labels]
)

fig.update_xaxes(range=[0,100])
fig.update_yaxes(range=[0,100])

fig.update_layout(
    title_text="United States Student Inflow/Outflow",
    barmode="stack",
    uniformtext=dict(mode="hide", minsize=10),
)

### Sankey

In [ ]:
# from_country -> source
# to_country -> target

In [178]:
from floweaver import *
import plotly.graph_objects as go

In [186]:
dw_in['source'] = 'United States'

In [288]:
source = list(dw_in[dw_in.year==2018].source)
target = list(dw_in[dw_in.year==2018].country)
value = list(dw_in[dw_in.year==2018].students)
value = [int(x) for x in value]

In [300]:
label = list(dw_in[dw_in.year==2018].country)
label.insert(0,'United States')

In [301]:
label

['United States',
 'China',
 'India',
 'South Korea',
 'Saudi Arabia',
 'Canada',
 'Vietnam',
 'Taiwan',
 'Japan',
 'Brazil',
 'Mexico',
 'Nigeria',
 'Nepal',
 'Iran',
 'United Kingdom',
 'Turkey',
 'Kuwait',
 'Germany',
 'France',
 'Indonesia',
 'Bangladesh',
 'Colombia',
 'Pakistan',
 'Venezuela',
 'Malaysia',
 'Spain']

In [292]:
# data
label = label
source = source
target = target
value = value
# data to dict, dict to sankey
link = dict(source = source, target = target, value = value)
node = dict(label = label, pad=5, thickness=2)
data = go.Sankey(link = link, node=node)
# plot
fig = go.Figure(data)
fig.show()


In [293]:
# data
label = ["ZERO", "ONE", "TWO", "THREE", "FOUR", "FIVE"]
source = [0, 0, 1, 1, 0]
target = [2, 3, 4, 5, 4]
value = [8, 2, 2, 8, 4]
# data to dict, dict to sankey
link = dict(source = source, target = target, value = value)
node = dict(label = label, pad=50, thickness=5)
data = go.Sankey(link = link, node=node)
# plot
fig = go.Figure(data)
fig.show()


In [ ]:
nodes = {
    'start': ProcessGroup(['Syria']), # one (Syria) at the start 
    'end': ProcessGroup(list(refugees['target'])), # 7 at the end
}

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c317a1b8-5df8-4084-a9ee-7f5cdd0a8b44' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>